<a href="https://colab.research.google.com/github/Kanghee-Lee/Mask-RCNN_TF/blob/master/Mask_RCNN(RPN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Imports
import numpy as np
import os
import glob
import cv2
!pip install xmltodict
import xmltodict
import tensorflow as tf
import math
from tqdm import tqdm
from PIL import Image, ImageDraw
from google.colab.patches import cv2_imshow
from keras import layers as layers
from keras import models as models

In [0]:

#download VOC data and extract .tar file
!mkdir train
!mkdir test
!wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar -P train/
!wget https://storage.googleapis.com/coco-dataset/external/PASCAL_VOC.zip -P train/
!wget http://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar -P test/
!tar -xf test/VOCtest_06-Nov-2007.tar -C test/
!tar -xf train/VOCtrainval_06-Nov-2007.tar -C train/
!unzip train/PASCAL_VOC.zip -d train/
!rm -rf train/PASCAL_VOC.zip train/VOCtrainval_06-Nov-2007.tar
!rm -rf test/VOCtest_06-Nov-2007.tar

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
data_images_path     = os.getcwd()+'/train/VOCdevkit/VOC2007/JPEGImages'
data_annotation_path = os.getcwd()+'/train/VOCdevkit/VOC2007/Annotations'
img_min=800
img_max=1024
image_height = 1024
image_width  = 1024
image_depth  = 3        # RGB
rpn_kernel_size = 3     # 3x3
subsampled_ratio = [4, 8, 16, 32, 64]    # Pooling 3 times
anchor_sizes = [32,64,128, 256, 512]      # Using [128, 256, 512] sizes in 1000x600 img size / Used [32, 64, 128] sizes in 224x224
anchor_aspect_ratio = [[1,1],[1/math.sqrt(2),math.sqrt(2)],[math.sqrt(2),1/math.sqrt(2)]]
num_anchors_in_box = len(anchor_aspect_ratio)
neg_threshold = 0.3
pos_threshold = 0.7
nms_threshold = 0.7
anchor_sampling_amount = 128        # 128 for each positive, negative sampling

In [0]:
list_images      = sorted([x for x in glob.glob(data_images_path + '/**')])    
#list_images=list_images[:2000]
total_images = len(list_images)
print(total_images)
list_annotations = sorted([x for x in glob.glob(data_annotation_path + '/**')]) 
#list_annotations = list_annotations[:2000]

# evaluating data consistency between images and annotations
t1=[]
t2=[]
for i in range(len(list_images)) :
    t1.append(list_images[i][-11:-4])
for i in range(len(list_annotations)) :
    t2.append(list_annotations[i][-11:-4])

for i in range(len(list_annotations)) :
    if t2[i] not in t1 :
        print(list_annotations[i])

print(len(list_annotations))


In [0]:
def get_classes(xml_files=list_annotations):
    '''
    Input : dataset's annotations
    parsing xml data to get objects label from images
    Output : class label
    
    '''
    classes = []
    
    for file in xml_files: 

        f = open(file)
        doc = xmltodict.parse(f.read()) #parse the xml file to python dict.
        # annotation - object - [obj1, obj2 , ...]
        try: 
            
            for obj in doc['annotation']['object']:
                classes.append(obj['name'].lower()) 
        # annotation - object
        except TypeError as e: 
            classes.append(doc['annotation']['object']['name'].lower()) 

        f.close()

    classes = list(set(classes)) 
    classes.sort()

    return classes

In [0]:
classes = get_classes(list_annotations)
print(classes)
num_of_class = len(classes)
print(num_of_class)

In [0]:
def get_labels_from_xml(xml_file_path, num_of_class = num_of_class):
    '''
    Input : 1 xml file
    Get class label, gt box coordinates.
    Because images are resized to 224x224, coordinates also need to be resized.
    Output: Existing class label, ground truth box coordinates
    '''

    f = open(xml_file_path)
    doc = xmltodict.parse(f.read()) 


    ori_img_height = float(doc['annotation']['size']['height'])
    ori_img_width  = float(doc['annotation']['size']['width'])


    class_label = [] 
    bbox_label  = [] 

    # multi-objects in image
    try:
        for each_obj in doc['annotation']['object']:
            obj_class = each_obj['name'].lower() 
            # Get bounding box coordinates
            x_min = float(each_obj['bndbox']['xmin']) # top left x-axis coordinate.
            x_max = float(each_obj['bndbox']['xmax']) # bottom right x-axis coordinate.
            y_min = float(each_obj['bndbox']['ymin']) # top left y-axis coordinate.
            y_max = float(each_obj['bndbox']['ymax']) # bottom right y-axis coordinate.

            # Images resized to 224x224. So resize the coordinates
            x_min = float((image_width/ori_img_width)*x_min)
            y_min = float((image_height/ori_img_height)*y_min)
            x_max = float((image_width/ori_img_width)*x_max)
            y_max = float((image_height/ori_img_height)*y_max)

            generated_box_info = [x_min, y_min, x_max, y_max]       # [top-left, bottom-right]

            index = classes.index(obj_class) 

            class_label.append(index)
            bbox_label.append(np.asarray(generated_box_info, dtype='float32'))

    # single-object in image
    except TypeError as e : 

        obj_class = doc['annotation']['object']['name']
        x_min = float(doc['annotation']['object']['bndbox']['xmin']) 
        x_max = float(doc['annotation']['object']['bndbox']['xmax']) 
        y_min = float(doc['annotation']['object']['bndbox']['ymin']) 
        y_max = float(doc['annotation']['object']['bndbox']['ymax']) 

        x_min = float((image_width/ori_img_width)*x_min)
        y_min = float((image_height/ori_img_height)*y_min)
        x_max = float((image_width/ori_img_width)*x_max)
        y_max = float((image_height/ori_img_height)*y_max)

        generated_box_info = [x_min, y_min, x_max, y_max]

        index = classes.index(obj_class) 


        class_label.append(index)
        bbox_label.append(np.asarray(generated_box_info, dtype='float32'))


    return class_label, np.asarray(bbox_label)
xml_file_path=list_annotations[3]

im = cv2.imread(list_images[3])
cv2_imshow(im)

class_label, bbox_label = get_labels_from_xml(xml_file_path, num_of_class = num_of_class)
print(class_label)
print(bbox_label)

In [0]:
def generate_anchors(one_anchor_size, one_subsampled_ratio,
                     rpn_kernel_size=rpn_kernel_size, anchor_aspect_ratio=anchor_aspect_ratio):

    '''
    Input : subsample_ratio (=Pooled ratio)
    * 1 anchor size, subsampled ratio are correnpondent to 3 anchor ratio *
    generate anchor in feature map. Then project it to original image.
    Output : list of anchors (x,y,w,h) and anchor_boolean (ignore anchor if value equals 0)

    '''

    list_of_anchors = []
    anchor_booleans = [] #This is to keep track of an anchor's status. Anchors that are out of boundary are meant to be ignored.

    starting_center = divmod(rpn_kernel_size, 2)[0] # rpn kernel's starting center in feature map
    
    anchor_center = [starting_center - 1,starting_center] # -1 on the x-coor because the increment comes first in the while loop
    
    subsampled_height = image_height/one_subsampled_ratio       # 
    subsampled_width = image_width/one_subsampled_ratio         # 
    
    while (anchor_center != [subsampled_width, subsampled_heigh]):  # != [26, 26] -> [N, N]

        anchor_center[0] += 1 #Increment x-axis

        #If sliding window reached last center, increase y-axis
        if anchor_center[0] > subsampled_width - (1 + starting_center):
            anchor_center[1] += 1
            anchor_center[0] = starting_center

        #anchors are referenced to the original image. 
        #Therefore, multiply downsampling ratio to obtain input image's center 
        anchor_center_on_image = [anchor_center[0]*one_subsampled_ratio, anchor_center[1]*one_subsampled_ratio]

        for size in [one_anchor_size]:
            print(size)
            for a_ratio in anchor_aspect_ratio:
                # [x,y,w,h] --> [y, x, h, w]
                anchor_info = [anchor_center_on_image[1], anchor_center_on_image[0], size*a_ratio[1], size*a_ratio[0]]
                print(anchor_info)
                # check whether anchor crosses the boundary of the image or not
                
                if (anchor_info[0] - anchor_info[2]/2 < 0 or anchor_info[0] + anchor_info[2]/2 > image_width or 
                                        anchor_info[1] - anchor_info[3]/2 < 0 or anchor_info[1] + anchor_info[3]/2 > image_height) :

                    anchor_booleans.append([0.0])       # if anchor crosses boundary, anchor_booleans=0

                else:

                    anchor_booleans.append([1.0])

                list_of_anchors.append(anchor_info)
    
    return list_of_anchors, anchor_booleans

def generate_pyramid_anchors(anchor_sizes=anchor_sizes, subsampled_ratio=subsampled_ratio,
                             rpn_kernel_size=rpn_kernel_size, anchor_aspect_ratio=anchor_aspect_ratio):
    list_of_anchors, anchor_booleans=[], []
    for i in range(len(anchor_sizes)) :
        anchors, bools = generate_anchors(one_subsampled_ratio=subsampled_ratio[i],
                                          one_anchor_size=anchor_sizes[i])
        list_of_anchors.append(anchors)
        anchor_booleans.append(bools)
    return np.concatenate(list_of_anchors, axis=0), np.concatenate(anchor_booleans)    # return all of anchors generated from pyramid feature network / shape : N*3
def normalize_box(anchors) :
    width=anchors[:,3]-anchors[:,1]
    height=anchors[:,2] - anchors[:,0]
    scale=np.array([h-1, w-1, h-1, w-1])
    shift=np.array([0, 0, 1, 1])
    return np.divide((anchors-shift), scale).astype(np.float32)

generate_pyramid_anchors()

In [0]:
import numpy as np
a=np.array([1, 2, 3])
b=np.array([1, 2, 3])
c=[]
c.append(a)
c.append(b)
print(np.concatenate(c))

In [0]:
def generate_label(class_labels, ground_truth_boxes, anchors, anchor_booleans, num_class=num_of_class,
                    neg_anchor_thresh = neg_threshold, pos_anchor_thresh = pos_threshold):
    '''
    Input  : classes, ground truth box (top-left, bottom-right), all of anchors, anchor booleans.
    Compute IoU to get positive, negative samples.
    if IoU > 0.7, positive / IoU < 0.3, negative / Otherwise, ignore
    Output : anchor booleans (to know which anchor to ignore), objectness label, regression coordinate in one image
    '''


    number_of_anchors = len(anchors) #Get the total number of anchors.

    anchor_boolean_array   = np.reshape(np.asarray(anchor_booleans),(number_of_anchors, 1))
    
    # IoU is more than threshold or not.
    objectness_label_array = np.zeros((number_of_anchors, 2), dtype=np.float32)
    # delta(x, y, w, h)
    box_regression_array   = np.zeros((number_of_anchors, 4), dtype=np.float32)
    # belongs to which object for every anchor
    class_array            = np.zeros((number_of_anchors, num_class), dtype=np.float32)
    
    for j in range(ground_truth_boxes.shape[0]):

        #Get the ground truth box's coordinates.
        gt_box_top_left_x = ground_truth_boxes[j][0]
        gt_box_top_left_y = ground_truth_boxes[j][1]
        gt_box_btm_rght_x = ground_truth_boxes[j][2]
        gt_box_btm_rght_y = ground_truth_boxes[j][3]

        #Calculate the area of the original bounding box.1 is added since the index starts from 0 not 1.
        gt_box_area = (gt_box_btm_rght_x - gt_box_top_left_x + 1)*(gt_box_btm_rght_y - gt_box_top_left_y + 1)

    
        for i in range(number_of_anchors):

            ######### Compute IoU #########

            # Check if the anchor should be ignored or not. If it is to be ignored, it crosses boundary of image.
            if int(anchor_boolean_array[i][0]) == 0:

                continue

            anchor = anchors[i] #Select the i-th anchor [x,y,w,h]

            #anchors are in [x,y,w,h] format, convert them to the [top-left-x, top-left-y, btm-right-x, btm-right-y]
            anchor_top_left_x = anchor[0] - anchor[2]/2
            anchor_top_left_y = anchor[1] - anchor[3]/2
            anchor_btm_rght_x = anchor[0] + anchor[2]/2
            anchor_btm_rght_y = anchor[1] + anchor[3]/2

            # Get the area of the bounding box.
            anchor_box_area = (anchor_btm_rght_x - anchor_top_left_x + 1)*(anchor_btm_rght_y - anchor_top_left_y + 1)

            # Determine the intersection rectangle.
            int_rect_top_left_x = max(gt_box_top_left_x, anchor_top_left_x)
            int_rect_top_left_y = max(gt_box_top_left_y, anchor_top_left_y)
            int_rect_btm_rght_x = min(gt_box_btm_rght_x, anchor_btm_rght_x)
            int_rect_btm_rght_y = min(gt_box_btm_rght_y, anchor_btm_rght_y)

            # if the boxes do not intersect, difference = 0
            int_rect_area = max(0, int_rect_btm_rght_x - int_rect_top_left_x + 1)*max(0, int_rect_btm_rght_y - int_rect_top_left_y)

            # Calculate the IoU
            intersect_over_union = float(int_rect_area / (gt_box_area + anchor_box_area - int_rect_area))
            
            # Positive
            if intersect_over_union >= pos_anchor_thresh:

                objectness_label_array[i][0] = 1.0 
                objectness_label_array[i][1] = 0.0 
                
                #get the class label
                class_label = class_labels[j]
                class_array[i][int(class_label)] = 1.0 #Denote the label of the class in the array.
                
                #Get the ground-truth box's [x,y,w,h]
                gt_box_center_x = ground_truth_boxes[j][0] + ground_truth_boxes[j][2]/2
                gt_box_center_y = ground_truth_boxes[j][1] + ground_truth_boxes[j][3]/2
                gt_box_width    = ground_truth_boxes[j][2] - ground_truth_boxes[j][0]
                gt_box_height   = ground_truth_boxes[j][3] - ground_truth_boxes[j][1]

                #Regression loss / weight
                delta_x = (gt_box_center_x - anchor[0])/anchor[2]
                delta_y = (gt_box_center_y - anchor[1])/anchor[3]
                delta_w = math.log(gt_box_width/anchor[2])
                delta_h = math.log(gt_box_height/anchor[3])

                box_regression_array[i][0] = delta_x
                box_regression_array[i][1] = delta_y
                box_regression_array[i][2] = delta_w
                box_regression_array[i][3] = delta_h

            if intersect_over_union <= neg_anchor_thresh:
                if int(objectness_label_array[i][0]) == 0:
                    objectness_label_array[i][1] = 1.0

            if intersect_over_union > neg_anchor_thresh and intersect_over_union < pos_anchor_thresh:
                if int(objectness_label_array[i][0]) == 0 and int(objectness_label_array[i][1]) == 0:
                    anchor_boolean_array[i][0] = 0.0 # ignore this anchor


    return anchor_boolean_array, objectness_label_array, box_regression_array, class_array

In [0]:
def anchor_sampling(anchor_booleans, objectness_label, anchor_sampling_amount=anchor_sampling_amount):

    '''
    Input : anchor booleans and objectness label
    fixed amount of negative anchors and positive anchors for training. 
    If we use all the neg and pos anchors, model will overfit on the negative samples.
    Output: Updated anchor booleans. 
    '''

    positive_count = 0
    negative_count = 0
    
    for i in range(objectness_label.shape[0]):
        if int(objectness_label[i][0]) == 1: #If the anchor is positive

            if positive_count > anchor_sampling_amount: #If the positive anchors are more than the threshold amount, set the anchor boolean to 0.

                anchor_booleans[i][0] = 0.0

            positive_count += 1

        if int(objectness_label[i][1]) == 1: #If the anchor is negatively labelled.
            if negative_count > anchor_sampling_amount: #If the negative anchors are more than the threshold amount, set the boolean to 0.

                anchor_booleans[i][0] = 0.0

            negative_count += 1
    
    return anchor_booleans


In [0]:
def generate_dataset(first_index, last_index, anchors, anchor_booleans):
        '''
        Input : starting index and final index of the dataset to be generated.
        Output: Anchor booleans, Objectness Label and Regression Label in batches.
        '''
        num_of_anchors = len(anchors)
        
        batch_anchor_booleans   = []
        batch_objectness_array  = []
        batch_regression_array  = []
        batch_class_label_array = []

        for i in range(first_index, last_index):

            #Get the true labels and the ground truth boxes [x,y,w,h] for every file.
            true_labels, ground_truth_boxes = get_labels_from_xml(xml_file_path=list_annotations[i])

            # generate_labels for specified batches
            anchor_bools, objectness_label_array, box_regression_array, class_array = generate_label(true_labels, ground_truth_boxes, 
                                                                                                        anchors, anchor_booleans)
            #ggenerate_label(class_labels, ground_truth_boxes, anchors, anchor_booleans, num_class=num_of_class,
            #        neg_anchor_thresh = neg_threshold, pos_anchor_thresh = pos_threshold)

            # get the updated anchor bools based on the fixed number of sample
            anchor_bools = anchor_sampling(anchor_bools, objectness_label_array)
            
            batch_anchor_booleans.append(anchor_bools)
            batch_objectness_array.append(objectness_label_array)
            batch_regression_array.append(box_regression_array)
            batch_class_label_array.append(class_array)

        batch_anchor_booleans   = np.reshape(np.asarray(batch_anchor_booleans), (-1,num_of_anchors))            # (1, 6084, 1) -> (1, 6084)
        
        batch_objectness_array  = np.asarray(batch_objectness_array)
        batch_regression_array  = np.asarray(batch_regression_array)
        batch_class_label_array = np.asarray(batch_class_label_array)

        return (batch_anchor_booleans, batch_objectness_array, batch_regression_array, batch_class_label_array)

In [0]:
def read_images(first_index, last_index):
    '''
    Read the image files, then resize.
    Input : first and last index.
    Output: numpy array of images.
    '''
    images_list = []
    
    for i in range(first_index, last_index):
        
        im = cv2.imread(list_images[i])
        im = cv2.resize(im, (image_height, image_width))/255
        
        images_list.append(im)
    
    return np.asarray(images_list)

In [0]:
anchors, an_bools = generate_anchors() #We only need to generate the anchors and the anchor booleans once.
num_of_anchors = len(anchors)

In [0]:
a,b,c,d = generate_dataset(0,1, anchors, an_bools)
a.shape
print(a)

In [0]:
learning_rate = 1e-5
epoch = 100
batch_size = 10
!mkdir -p '/content/drive/My Drive/VOCdata/trained_weight'
model_checkpoint = './drive/My Drive/VOCdata/trained_weight/model.ckpt'
decay_steps = 10000
decay_rate = 0.99
lambda_value = 10

In [0]:
def smooth_func(t):
    
    t = tf.abs(t)
    
    comparison_tensor = tf.ones((num_of_anchors, 4))
    smoothed = tf.where(tf.less(t, comparison_tensor), 0.5*tf.pow(t,2), t - 0.5)
    
    return smoothed

In [0]:

def smooth_L1(pred_box, truth_box):
    
    diff = pred_box - truth_box
    
    smoothed = tf.map_fn(smooth_func, diff)
    
    return smoothed

In [0]:
def identity_block(input, filter_size, filter_num) :
    y=layers.Conv2D(filter_num[0], (1, 1), strides=(1, 1), padding='valid', use_bias=True)(input)
    y=layers.BatchNormalization(y)
    y=layers.Activation('relu')(y)

    y=layers.Conv2D(filter_num[1], (3, 3), strides=(1, 1), padding='same', use_bias=True)(y)
    y=layers.BatchNormalization(y)
    y=layers.Activation('relu')(y)

    y=layers.Conv2D(filter_num[2], (1, 1), strides=(1, 1), padding='valid', use_bias=True)(y)
    y=layers.BatchNormalization(y)
    
    y=layers.Add()([y, input])
    y=layers.Activation('relu')(y)
    return y

def conv_block(input, filter_num, strides=(2, 2)) :
    y=layers.Conv2D(filter_num[0], (1, 1), strides=strides, padding='valid', use_bias=True)(input)
    y=layers.BatchNormalization(y)
    y=layers.Activation('relu')(y)

    y=layers.Conv2D(filter_num[1], (3, 3), strides=(1, 1), padding='same', use_bias=True)(y)
    y=layers.BatchNormalization(y)
    y=layers.Activation('relu')(y)

    y=layers.Conv2D(filter_num[2], (1, 1), strides=(1, 1), padding='valid', use_bias=True)(y)
    y=layers.BatchNormalization(y)
    # add shortcut block for conv_block
    shortcut=layers.Conv2D(filter_num[2], (1, 1), strides=strides, padding='valid', use_bias=True)(input)
    shortcut=layers.BatchNormalization(shortcut)
    
    y=layers.Add()([y, shortcut])
    y=layers.Activation('relu')(shortcut)
    return y

def resnet101(input) :
    # stage 1
    y=layers.ZeroPadding2D((3, 3))(input)
    y=layers.Conv2D(64, (7, 7), strides=(2, 2), padding='valid', use_bias=True)(y)
    y=layers.BatchNormalization(y)
    y=layers.Activation('relu')(y)
    F1=y=layers.MaxPooling2D((3, 3), strides(2, 2), padding='same')(y)

    # stage 2 : In stage 2, use strides 1
    y=conv_block(y, [64, 64, 256], strides=(1, 1))
    y=identity_block(y, [64, 64, 256])    
    F2=y=identity_block(y, [64, 64, 256])

    # stage 3 
    y=conv_block(y, [128, 128, 512])
    y=identity_block(y, [128, 128, 256])    
    y=identity_block(y, [128, 128, 256])
    F3=y=identity_block(y, [128, 128, 256])

    # stage 4 : In stage 4, there are 22 identity blocks
    y=conv_block(y, [256, 256, 1024])
    block_num=22
    for i in range(block_num) :
        y=identity_block(y, [256, 256, 1024])
    F4=y
    
    # stage 5 
    y=conv_block(y, [512, 512, 2048])
    y=identity_block(y, [512, 512, 2048])
    F5=y=identity_block(y, [512, 512, 2048])

    return [F1, F2, F3, F4, F5]

# Use 256 filters for FPN
def fpn(input) :
    P5=layers.Conv2d(256, (1, 1))(F5)
    P5=layers.Conv2D(256, (3, 3), padding='same')(P5)
    # Compute P4 by adding feature map4 and Upsampled result of P5
    res4=layers.UpSammpling2D(size=(2, 2))(P5)
    P4=layers.Add([res4, layers.Conv2D(256, (1, 1)(F4))])
    P4=layers.Conv2D(256, (3, 3), padding='same')(P4)

    res3=layers.UpSammpling2D(size=(2, 2))(P4)
    P3=layers.Add([res4, layers.Conv2D(256, (1, 1)(F3))])
    P3=layers.Conv2D(256, (3, 3), padding='same')(P3)

    res2=layers.UpSammpling2D(size=(2, 2))(P3)
    P2=layers.Add([res4, layers.Conv2D(256, (1, 1)(F2))])
    P2=layers.Conv2D(256, (3, 3), padding='same')(P2)

    #Issues : Why use (1, 1) for pool size and reason for not computing (3, 3) conv only in P6
    P6=layers.MaxPooling2D(pool_size=(1, 1), strides=2)(P5)

    return [P2, P3, P4, P5, P6]

In [0]:
def rpn(input) :
    # All of feature map's depth in FPN is 256
    input_featuremap=layers.Input(shape=[None, None, 256])
    shared4obj_and_bb=layers.Conv2d(512, (3, 3), padding='same', activation_fn='relu', strides=1)(input)
    obj_conv = layers.Conv2d(6, (1, 1), padding='VALID', activation_fn='linear')(shared4obj_and_bb)         # None * None * 6
    bb_conv = layers.Conv2d(12, (1, 1), padding='VALID', activation_fn='linear')(shared4obj_and_bb)          # None * None * 12

    class_conv_reshape = tf.reshape(obj_conv, (-1, num_of_anchors, 2))          # 6084x2
    anchor_conv_reshape = tf.reshape(bb_conv, (-1, num_of_anchors, 4))          # 6084x4

    logits = tf.nn.softmax(class_conv_reshape)
    
    return models.Model([input_featuremap], [class_conv_reshape, logits, anchor_conv_reshape])

In [0]:
def apply_box_delta(boxes, delta) : 
    '''
    Input : boxes [N, [y1, x1, y2, x2]]
    boxes = anchor boxes (ranked top k of All anchors) 
    delta = correspondents to anchor boxes' delta (also ranked top k anchors' delta)

    apply delta to boxes (* refined boxes' coordinates represent normalized coordinates)
    '''
    y1, x1, y2, x2 = boxes[:,0], boxes[:,1], boxes[:,2], boxes[:,3]
    anchor_height = y2 - y1
    anchor_width = x2 - x1
    anchor_center_x = x1 + width*0.5
    anchor_center_y = y1 + height*0.5
    # apply delta to box
    refined_center_x = delta[:,1] * width + anchor_center_x
    refined_center_y = delta[:,0] * height + anchor_center_y
    refined_width = tf.exp(delta[:,3]) * anchor_width
    refined_height = tf.exp(delta[:,2]) * anchor_height

    # compute refined coordinates : [y1, x1, y2, x2]
    refined_x1=refined_center_x - 0.5*width
    refined_x2=refined_center_x + 0.5*width
    refined_y1=refined_center_y - 0.5*height
    refined_y2=refined_center_y + 0.5*height

    return tf.stack([refined_y1, refined_x1, refined_y2, refined_x2], axis=1)

In [0]:
def compute_iou_boxes(one_box, boxes, box_area, boxes_area) :
    intersection_y1 = np.maximum(box[0], boxes[:, 0])
    intersection_x1 = np.maximum(box[1], boxes[:, 1])
    intersection_y2 = np.minimum(box[2], boxes[:, 2])
    intersection_x2 = np.minimum(box[3], boxes[:, 3])

    inter_area = np.maximum(intersection_x2-intersection_x1, 0) * np.maximum(intersection_y1-intersection_y2, 0)
    union_area = box_area + boxes_area[:] - inter_area
    iou = inter_area / union_area
    return iou

def non_max_suppression(boxes, scores) : 
    # scores : result from softmax of rpn's output(logits)
    y1=boxes[:, 0]
    x1=boxes[:, 1]
    y2=boxes[:, 2]
    x2=boxes[:, 3]
    area=(y2-y1)*(x2-x1)

    # Use fg score to sort
    sorted_idx = scores.argsort()[::-1]         # return idx of sorted result
    
    result = []
    while len(ixs) > 0:
        # Pick top box and add its index to the list
        i = sorted_idx[0]
        pick.append(i)
        # Compute IoU of the picked box with the rest
        iou = compute_iou(boxes[i], boxes[ixs[1:]], area[i], area[ixs[1:]])
        # Identify boxes with IoU over the 0.7(nms_threshold). 
        # This returns are correspondents to ixs[1:], so add 1 (ixs are correspondents to ixis[:]).
        remove_ixs = np.where(iou > nms_threshold)[0] + 1
        # Remove indices of the picked and overlapped boxes.
        ixs = np.delete(ixs, remove_ixs)
        ixs = np.delete(ixs, 0)
    nms_roi = tf.gather(boxes, result)
    return nms_roi

In [0]:
# Loss Function
loss1 = 1/256*tf.reduce_sum(anch_bool*(tf.nn.softmax_cross_entropy_with_logits(labels=Y_obj, logits=class_conv_reshape)))       # positive(128) + negative(128)
# (10, 6084, 2)
loss2 = lambda_value*(1/128)*tf.reduce_sum((tf.reshape(Y_obj[:,:,0], (-1,num_of_anchors,1)))*smooth_L1(anchor_conv_reshape, Y_coor))

total_loss = loss1 + loss2

optimizer = tf.train.AdamOptimizer(decayed_lr).minimize(total_loss, global_step=global_step)

global_step = tf.Variable(0, trainable=False)
decayed_lr = tf.train.exponential_decay(learning_rate,
                                            global_step, decay_steps,
                                            decay_rate, staircase=True)


In [0]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

try:
    saver.restore(sess, model_checkpoint)
    print("Model has been loaded!")
    
except:
    
    print("Model doens't exist!")

In [0]:
'''
def draw_a_rectangel_in_img(draw_obj, box, color, width):
    '''
    use draw lines to draw rectangle. since the draw_rectangle func can not modify the width of rectangle
    :param draw_obj:
    :param box: [x1, y1, x2, y2]
    :return:
    '''
    x1, y1, x2, y2 = box[0], box[1], box[2], box[3]
    top_left, top_right = (x1, y1), (x2, y1)
    bottom_left, bottom_right = (x1, y2), (x2, y2)

    draw_obj.line(xy=[top_left, top_right],
                  fill=color,
                  width=width)
    draw_obj.line(xy=[top_left, bottom_left],
                  fill=color,
                  width=width)
    draw_obj.line(xy=[bottom_left, bottom_right],
                  fill=color,
                  width=width)
    draw_obj.line(xy=[top_right, bottom_right],
                  fill=color,
                  width=width)
PIXEL_MEAN = [123.68, 116.779, 103.939]
def draw_boxes_with_label_and_scores(img_array, boxes):

    img_array = img_array + np.array(PIXEL_MEAN)
    img_array.astype(np.float32)
    boxes = boxes.astype(np.int64)
    #labels = labels.astype(np.int32)
    img_array = np.array(img_array * 255 / np.max(img_array), dtype=np.uint8)
    
    img_obj = Image.fromarray(img_array)
    #img_obj=img_array
    print('zz')
    raw_img_obj = img_obj.copy()
    
    draw_obj = ImageDraw.Draw(img_obj)
    num_of_objs = 0
    for box in boxes:
         draw_a_rectangel_in_img(draw_obj, box, color='Coral', width=3)
        

    out_img_obj = Image.blend(raw_img_obj, img_obj, alpha=0.6)

    return np.array(out_img_obj)
'''
#TRAINING 

for epoch_idx in range(epoch): #Each epoch.
    
    #Loop through the whole dataset in batches.
    for start_idx in tqdm(range(0, total_images, batch_size)):
        
        end_idx = start_idx + batch_size
        
        if end_idx >= total_images : end_idx = total_images - 1 #In case the end index exceeded the dataset.
            
        images = read_images(start_idx, end_idx) #Read images.
        
        #Get the labels needed.
        batch_anchor_booleans, batch_objectness_array, batch_regression_array, _ = \
                                                generate_dataset(start_idx,end_idx, anchors, an_bools)
        print(batch_objectness_array.shape)
        #Optimize the model.
        anchor_reshape, _, theloss = sess.run([anchor_conv_reshape, optimizer, total_loss], feed_dict={X: images,
                                                                  Y_obj:batch_objectness_array,
                                                                  Y_coor: batch_regression_array,
                                                                  anch_bool: batch_anchor_booleans})
        
        '''
        img_array = cv2.imread(list_images[0])
            #img_array = cv2.resize(img_array, (image_height, image_width))/255
        img_array = np.array(img_array, np.float32) - np.array(PIXEL_MEAN)

        anchor_booleans, objectness_array, regression_array, _ = \
                                                generate_dataset(0,1, anchors, an_bools)
        img_array_tensor=read_images(0, 1)
        anchor_reshape=sess.run([anchor_conv_reshape], feed_dict={X:img_array_tensor, Y_obj : objectness_array, Y_coor : regression_array, anch_bool : anchor_booleans})
        print(anchor_reshape)
        print(anchor_reshape[0])
        
        anchor_reshape=anchor_reshape[0]
        boxes = np.array(
        [[200, 200, 500, 500],
            [300, 300, 400, 400],
            [200, 200, 400, 400]]
    )
        anchor_reshape=np.array(anchor_reshape)
        
        im=draw_boxes_with_label_and_scores(img_array, np.array(anchor_reshape))
        cv2_imshow(im)        
        '''
    #Save the model periodically.
        saver.save(sess, model_checkpoint)
    
    print("Epoch : %d, Loss : %g"%(epoch_idx, theloss))
    img_array = cv2.imread(list_images[0])
        #img_array = cv2.resize(img_array, (image_height, image_width))/255
    img_array = np.array(img_array, np.float32) - np.array(PIXEL_MEAN)

    anchor_booleans, objectness_array, regression_array, _ = \
                                            generate_dataset(0,1, anchors, an_bools)
    img_array_tensor=read_images(0, 1)
    anchor_reshape=sess.run([anchor_conv_reshape], feed_dict={X:img_array_tensor, Y_obj : objectness_array, Y_coor : regression_array, anch_bool : anchor_booleans})
    print(anchor_reshape)
    print(anchor_reshape[0])
    anchor_reshape=anchor_reshape[0][0]
    boxes = np.array(
    [[200, 200, 500, 500],
        [300, 300, 400, 400],
        [200, 200, 400, 400]]
)
    im=draw_boxes_with_label_and_scores(img_array, np.array(anchor_reshape))
    cv2_imshow(im)